In [1]:
import numpy as np
import pandas as pd 
data = pd.read_csv("Prices.csv") 
from gurobipy import *

# Q1

## MODEL 1

In [2]:
m=4
df=data[["MSFT","GS","PG","SCHP"]]
dfarray=np.array(df)
mat=(dfarray[1:61]-dfarray[0:60])/df[0:60]
mean=np.mean(mat)
covmat=np.cov(np.transpose(mat))

mod = Model()
w = mod.addVars(m)




for i in range(m):
    mod.addConstr(w[i]>=0)


mod.addConstr(sum(w[i]*mean[i] for i in range(m))>=0.005)
    

mod.addConstr(sum(w[i] for i in range(m))==1) 


   

# Construct objective
mod.setObjective(sum(w[i]*w[j]*covmat[i][j] for i in range(m) for j in range(m)), GRB.MINIMIZE)
mod.update()
mod.optimize()
name=df.columns
for i in range(m):
    print ('The weight for',name[i],'is',w[i].x,'.')

print('The optimal value is',mod.objVal,',and the runtime is', mod.Runtime,'.')

Academic license - for non-commercial use only
Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.87864836e+05 -1.87864836e+05  3.50e+03 1.91e-07  1.00e+06     0s
   1   5.03195868e+03 -5.16248579e+03  2.59e+02 1.41e-08  7.60e+04     0s
   2   8.65509708e-03 -1.63980611e+02  5.37e-01 2.93e-11  1

## MODEL 2

In [22]:
m=390

dfarray=np.array(data)
mat2=(dfarray[1:61]-dfarray[0:60])/data[0:60]
mean=np.mean(mat2)
covmat=np.cov(np.transpose(mat2))

mod = Model()
w = mod.addVars(m)





for i in range(m):
    mod.addConstr(w[i]>=0)


mod.addConstr(sum(w[i]*mean[i] for i in range(m))>=0.005)
    

mod.addConstr(sum(w[i] for i in range(m))==1) 
# each room needs to be assigned exactly once 

    

# Construct objective
mod.setObjective(sum(w[i]*w[j]*covmat[i][j] for i in range(m) for j in range(m)), GRB.MINIMIZE)
mod.update()
mod.optimize()
print('The optimal value is',mod.objVal,',and the runtime is', mod.Runtime,'.')

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.03s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.89821559e-13 -2.89821559e-13  3.90e+05 3.22e-07  1.00e+06     0s
   1   2.43039969e+03 -3.41875696e+03  1.04e+04 8.59e-09  2.67e+04     0s
   2   3.54479277e-03 -1.01970099e+03  9.49e+00 7.84e-12  2.70e+01     0s
   3   9.5698

## MODEL 3

In [23]:
m=390

dfarray=np.array(data)
mat2=(dfarray[1:61]-dfarray[0:60])/data[0:60]
mean=np.mean(mat2)
covmat=np.cov(np.transpose(mat2))

mod = Model()
w = mod.addVars(m)
x=mod.addVars(m,vtype = GRB.BINARY)


mod.addConstr(sum(x[i] for i in range(m))<=4)

for i in range(m):
    mod.addConstr(w[i]>=0)
    mod.addConstr(w[i] == w[i]*x[i])

mod.addConstr(sum(w[i]*mean[i]*x[i] for i in range(m))>=0.005)
    

mod.addConstr(sum(w[i] for i in range(m))==1)

# each room needs to be assigned exactly once 

    

# Construct objective
mod.setObjective(sum(w[i]*w[j]*covmat[i][j] for i in range(m) for j in range(m)), GRB.MINIMIZE)
mod.update()
mod.optimize()

name=data.columns
for i in range(m):
    if x[i].x==1:
        print ('The weight for',name[i],'is',w[i].x,'.')
        
print('The optimal value is',mod.objVal,',and the runtime is', mod.Runtime,'.')

Optimize a model with 392 rows, 780 columns and 1170 nonzeros
Model has 76245 quadratic objective terms
Model has 391 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-06, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
  QRHS range       [5e-03, 5e-03]
Presolve removed 390 rows and 0 columns
Presolve time: 0.02s
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0018152

Root relaxation: objective 2.878501e-05, 5052 iterations, 0.57 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0

# Q2

## (a) 
The optimal risk in Model 1 higher than Model 2, because Model 1 is based on 4 stocks while Model 2 is based on 390 stocks, which gives it more choices to balance out the risk, hence a lower optimal risk.

## (b) 
The optimal risk in Model 2 lower than Model 3, because Model 3 is only chooses 4 stocks out of 390 stocks while Model 2 is based on 390 stocks, which gives Model 2 more choices to balance out the risk, hence a lower optimal risk.

# Q3

## (a)

In [24]:
m=390

dfarray=np.array(data)
mat2=(dfarray[1:61]-dfarray[0:60])/data[0:60]
mean=np.mean(mat2)
covmat=np.cov(np.transpose(mat2))

mod = Model()
w = mod.addVars(m)
x=mod.addVars(m,vtype = GRB.BINARY)


mod.addConstr(sum(x[i] for i in range(m))<=4)

for i in range(m):
    mod.addConstr(w[i]>=0)
    mod.addConstr(w[i] == w[i]*x[i])

mod.addConstr(sum(w[i]*mean[i]*x[i] for i in range(m))>=0.005)
    

mod.addConstr(sum(w[i] for i in range(m))==1)

# each room needs to be assigned exactly once 

    
mod.Params.TimeLimit = 30.0
# Construct objective
mod.setObjective(sum(w[i]*w[j]*covmat[i][j] for i in range(m) for j in range(m)), GRB.MINIMIZE)
mod.update()
mod.optimize()

name=data.columns
for i in range(m):
    if x[i].x==1:
        print ('The weight for',name[i],'is',w[i].x,'.')
        
print('The optimal value is',mod.objVal,',and the runtime is', mod.Runtime,'.')

Changed value of parameter TimeLimit to 30.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 392 rows, 780 columns and 1170 nonzeros
Model has 76245 quadratic objective terms
Model has 391 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-06, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
  QRHS range       [5e-03, 5e-03]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0018152

Root relaxation: objective 2.878501e-05, 5052 iterations, 0.46 seconds

    Nodes    |    Current Node    |     Objective Bounds      |    

The objective function value at termination is the same compared with the optimal value obtained in question 1c.

## (b)

In [25]:
m=390

dfarray=np.array(data)
mat2=(dfarray[1:61]-dfarray[0:60])/data[0:60]
mean=np.mean(mat2)
covmat=np.cov(np.transpose(mat2))

mod = Model()
w = mod.addVars(m)
x=mod.addVars(m,vtype = GRB.BINARY)


mod.addConstr(sum(x[i] for i in range(m))<=4)

for i in range(m):
    mod.addConstr(w[i]>=0)
    mod.addConstr(w[i] == w[i]*x[i])

mod.addConstr(sum(w[i]*mean[i]*x[i] for i in range(m))>=0.005)
    

mod.addConstr(sum(w[i] for i in range(m))==1)

# each room needs to be assigned exactly once 

    
mod.Params.MIPGap = 0.1
# Construct objective
mod.setObjective(sum(w[i]*w[j]*covmat[i][j] for i in range(m) for j in range(m)), GRB.MINIMIZE)
mod.update()
mod.optimize()

name=data.columns
for i in range(m):
    if x[i].x==1:
        print ('The weight for',name[i],'is',w[i].x,'.')
        
print('The optimal value is',mod.objVal,',and the runtime is', mod.Runtime,'.')

Changed value of parameter MIPGap to 0.1
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Optimize a model with 392 rows, 780 columns and 1170 nonzeros
Model has 76245 quadratic objective terms
Model has 391 quadratic constraints
Variable types: 390 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-06, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
  QRHS range       [5e-03, 5e-03]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1563 rows, 1170 columns, 4680 nonzeros
Presolved model has 76245 quadratic objective terms
Variable types: 780 continuous, 390 integer (390 binary)
Found heuristic solution: objective 0.0018152

Root relaxation: objective 2.878501e-05, 5052 iterations, 0.44 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

The solver time is much smaller (roughly a half in this case) compared with the solution time obtained in question 1c).